In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting

In [2]:
order_products=pd.read_csv('/Users/jinglinhui/Desktop/Data_analytics_project/rawdata/order_products.csv')
orders=pd.read_csv('/Users/jinglinhui/Desktop/Data_analytics_project/rawdata/orders.csv')

## Group customer by their max order number

In [3]:
df=orders.groupby('user_id')['order_number'].aggregate('max').reset_index()

In [4]:
df['eval_set']=0

In [5]:
df.head(10)

,user_id,order_number,eval_set
0,1,11,0
1,2,15,0
2,3,13,0
3,4,6,0
4,5,5,0
5,6,4,0
6,7,21,0
7,8,4,0
8,9,4,0
9,10,6,0


## Joining grouped table to orders table by user_id,order_number

In [6]:
df1=pd.merge(orders,df, how='left', left_on=['user_id','order_number'], right_on=['user_id','order_number'])

In [7]:
df1.head(10)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,NaN
1,2398795,1,2,3,7,15.0,NaN
2,473747,1,3,3,12,21.0,NaN
3,2254736,1,4,4,7,29.0,NaN
4,431534,1,5,4,15,28.0,NaN
5,3367565,1,6,2,7,19.0,NaN
6,550135,1,7,1,9,20.0,NaN
7,3108588,1,8,1,14,14.0,NaN
8,2295261,1,9,1,16,0.0,NaN
9,2550362,1,10,4,8,30.0,NaN


## Fill in 'Prior' for the null value as they are not the lastest order

In [8]:
df1.eval_set.fillna('Prior',inplace=True)
df1

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,Prior
1,2398795,1,2,3,7,15.0,Prior
2,473747,1,3,3,12,21.0,Prior
3,2254736,1,4,4,7,29.0,Prior
4,431534,1,5,4,15,28.0,Prior
...,...,...,...,...,...,...,...
3421078,2266710,206209,10,5,18,29.0,Prior
3421079,1854736,206209,11,4,10,30.0,Prior
3421080,626363,206209,12,1,12,18.0,Prior
3421081,2977660,206209,13,1,12,7.0,Prior


## Join distinct order_products on to df1  table

In [40]:
distinct_order_id=order_products[['order_id']].drop_duplicates()
distinct_order_id

,order_id
0,2
9,3
17,4
30,5
56,6
...,...
33819080,3421049
33819086,3421056
33819091,3421058
33819099,3421063


In [51]:
distinct_order_products=pd.merge(distinct_order_id,order_products,on='order_id')
distinct_order_products

,order_id,product_id,add_to_cart_order
0,2,33120,1
1,2,28985,2
2,2,9327,3
3,2,45918,4
4,2,30035,5
...,...,...,...
33819101,3421063,14233,3
33819102,3421063,35548,4
33819103,3421070,35951,1
33819104,3421070,16953,2


In [74]:
join=pd.merge(df1,distinct_order_products,how='left',on='order_id')
join

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set,product_id,add_to_cart_order
0,2539329,1,1,2,8,NaN,Prior,196.0,1.0
1,2539329,1,1,2,8,NaN,Prior,14084.0,2.0
2,2539329,1,1,2,8,NaN,Prior,12427.0,3.0
3,2539329,1,1,2,8,NaN,Prior,26088.0,4.0
4,2539329,1,1,2,8,NaN,Prior,26405.0,5.0
...,...,...,...,...,...,...,...,...,...
33894101,272231,206209,14,6,14,30.0,0.0,40603.0,4.0
33894102,272231,206209,14,6,14,30.0,0.0,15655.0,5.0
33894103,272231,206209,14,6,14,30.0,0.0,42606.0,6.0
33894104,272231,206209,14,6,14,30.0,0.0,37966.0,7.0


## Creating table for prior, test and train

In [77]:
prior=join[join['eval_set']=="Prior"]
prior

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set,product_id,add_to_cart_order
0,2539329,1,1,2,8,NaN,Prior,196.0,1.0
1,2539329,1,1,2,8,NaN,Prior,14084.0,2.0
2,2539329,1,1,2,8,NaN,Prior,12427.0,3.0
3,2539329,1,1,2,8,NaN,Prior,26088.0,4.0
4,2539329,1,1,2,8,NaN,Prior,26405.0,5.0
...,...,...,...,...,...,...,...,...,...
33894093,2977660,206209,13,1,12,7.0,Prior,14197.0,5.0
33894094,2977660,206209,13,1,12,7.0,Prior,38730.0,6.0
33894095,2977660,206209,13,1,12,7.0,Prior,31477.0,7.0
33894096,2977660,206209,13,1,12,7.0,Prior,6567.0,8.0


## Check matching after the join and the order is latest

In [76]:
train = join[(join['product_id'].notnull()) & (join['eval_set']==0)]
train.loc[train.eval_set == 0, 'eval_set'] = 'Train'

## Check orders didn't match in the lastest order

In [75]:
test = join[join['product_id'].isnull()]
test.loc[test.eval_set == 0, 'eval_set'] = 'Test'

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## Join all the tables

In [78]:
new_order=pd.concat([prior,test,train], axis=0)

In [79]:
new_order=new_order.drop(['product_id','add_to_cart_order'],axis=1)

In [80]:
new_order

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,eval_set
0,2539329,1,1,2,8,NaN,Prior
1,2539329,1,1,2,8,NaN,Prior
2,2539329,1,1,2,8,NaN,Prior
3,2539329,1,1,2,8,NaN,Prior
4,2539329,1,1,2,8,NaN,Prior
...,...,...,...,...,...,...,...
33894101,272231,206209,14,6,14,30.0,Train
33894102,272231,206209,14,6,14,30.0,Train
33894103,272231,206209,14,6,14,30.0,Train
33894104,272231,206209,14,6,14,30.0,Train


In [81]:
new_order['eval_set'].value_counts()

Prior    32434489
Train     1384617
Test        75000
Name: eval_set, dtype: int64